In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import warnings
warnings.simplefilter("ignore") 
import logging
logging.basicConfig(level=logging.ERROR)

In [4]:
from kinoml.datasets.chembl import ChEMBLDatasetProvider
chembl = ChEMBLDatasetProvider.from_source(sample=20000)

RDKit WARNING: [13:45:44] Enabling RDKit 2019.09.2 jupyter extensions
100%|██████████| 20000/20000 [00:00<00:00, 31661.17it/s]


In [5]:
chembl

<ChEMBLDatasetProvider with 20000 IC50Measurement measurements and 19150 systems>

In [6]:
df = chembl.to_dataframe()
df

,Systems,n_components,Measurement,MeasurementType
0,P16234 & COc1cc2nccc(Oc3ccc(NC(=O)Nc4c(F)cccc4...,2,3.5,IC50Measurement
1,Q08881 & Cc1ccc2cc(-c3n[nH]c4ccc(NC(=O)c5ccco5...,2,530.0,IC50Measurement
2,P17252 & Cc1cc(C2CCN(CCCF)CC2)cc2[nH]c(-c3c(NC...,2,11000.0,IC50Measurement
3,P12931 & COc1cc(Nc2nn3c(N[C@@H](CO)C(C)C)cc(C)...,2,432.0,IC50Measurement
4,P31749 & NC1(c2ccc(-c3nc4c(-c5cn[nH]c5)cccn4c3...,2,99.0,IC50Measurement
...,...,...,...,...
19995,P24941 & CNC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)c1cc(...,2,300.0,KdMeasurement
19996,P00519 & O=C(c1ccc(/C=C/c2n[nH]c3ccccc23)cc1)N...,2,24.0,KdMeasurement
19997,P42681 & COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1C...,2,3700.0,KdMeasurement
19998,Q9NRP7 & C=CC(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc...,2,4500.0,KdMeasurement


In [7]:
print("Measurements:", len(chembl.measurements))
print("Systems:", len(chembl.systems))
print("Proteins:", len(set([s.protein for s in chembl.systems])))
print("Ligands:",len(set([s.ligand.name for s in chembl.systems])))

Measurements: 20000
Systems: 19150
Proteins: 384
Ligands: 16523


Having this many ligands (compared to PKIS2) makes this dataset take much more memory and longer (~10 mins) to initialize!

In [8]:
from kinoml.features.ligand import SmilesToLigandFeaturizer, MorganFingerprintFeaturizer
from kinoml.features.core import HashFeaturizer, Concatenated, Pipeline

morgan_featurizer = Pipeline([SmilesToLigandFeaturizer(), MorganFingerprintFeaturizer(nbits=1024, radius=2)])
hashed_sequence_featurizer = HashFeaturizer(("protein", "sequence"))
concat_featurizers = Concatenated([morgan_featurizer, hashed_sequence_featurizer], axis=0)

You can prefeaturize everything before the loop with this cell:

In [9]:
# %%time
# chembl.featurize(concat_featurizers)

Or delay the featurization until the systems are needed by passing the featurizer to the `to_pytorch` constructor. We will use this strategy!

In [16]:
datasets = chembl.to_pytorch(featurizer=concat_featurizers)
datasets

In [17]:
observation_models = chembl.observation_models(backend="pytorch")
observation_models

[<function kinoml.core.measurements.IC50Measurement._observation_model_pytorch(values, substrate_conc=1e-06, michaelis_constant=1, inhibitor_conc=1e-06, **kwargs)>,
 <function kinoml.core.measurements.KiMeasurement._observation_model_pytorch(values, inhibitor_conc=1e-06, **kwargs)>,
 <function kinoml.core.measurements.KdMeasurement._observation_model_pytorch(values, inhibitor_conc=1e-06, **kwargs)>]

In [38]:
import torch
from kinoml.ml.models import NeuralNetworkRegression
from tqdm.auto import trange, tqdm

# precompute input size
input_size = datasets[0].estimate_input_size()
model = NeuralNetworkRegression(input_size=input_size[0])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_function = torch.nn.MSELoss() # Mean squared error

# Use DataLoader for minibatches
datasets = chembl.to_pytorch(featurizer=concat_featurizers)
loaders = [dataset.as_dataloader(batch_size=512) for dataset in datasets]
loaders

nb_epoch = 100
loss_timeseries = []
for epoch in trange(nb_epoch, desc="Epochs"):
    cumulative_loss = torch.zeros(1, dtype=torch.float64)
    for loader in loaders:
        for x, y in tqdm(loader, desc=f"Batches for {loader.dataset.observation_model.__qualname__.split('.')[0]}", leave=False):
            # Clear gradients
            optimizer.zero_grad()
            # Obtain model prediction given model input
            delta_g = model(x)

            # with observation model
            prediction = loader.dataset.observation_model(delta_g)
            loss = loss_function(prediction, y)

            # Obtain loss for the predicted output
            cumulative_loss[0] += loss.item()

            # Gradients w.r.t. parameters
            loss.backward()

            # Optimizer
            optimizer.step()
    loss_timeseries.append(cumulative_loss[0])
    if epoch % 5 == 0:
        print(f"epoch {epoch} : loss {loss_timeseries[-1]:.4E}")
print("Done!")

RDKit WARNING: [14:26:34] Conflicting single bond directions around double bond at index 14.
RDKit WARNING: [14:26:34]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [14:26:34] Conflicting single bond directions around double bond at index 14.
RDKit WARNING: [14:26:34]   BondStereo set to STEREONONE and single bond directions set to NONE.


epoch 0 : loss NAN


epoch 5 : loss NAN


epoch 10 : loss NAN


epoch 15 : loss NAN


...

epoch 80 : loss NAN


epoch 85 : loss NAN


epoch 90 : loss NAN


epoch 95 : loss NAN



Done!
